<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Diabetes prediction using Decision Forest Classifier and GLM</b>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Introduction</b></p>

<p style = 'font-size:16px;font-family:Arial'>According to research, there are a lot of people that are suffering from diabetes all over the world. Studies show that in 2019, diabetes was the direct cause of 1.5 million deaths and almost 50% of all deaths occurred before the age of 70.
<br>
<br>  
Over time, diabetes can have a negative impact on multiple organs. It can damage the heart, blood vessels, eyes, kidneys, and nerves. The earlier a person receives a proper treatment, the more likely he or she will be in lowering blood glucose level. In addition, the risk of failure of other organs will also be reduced.
<br>
<br>  
Hence, we are showcasing the complete approach about how we can make prediction of diagnosis of  diabetes 6 months in advance. We are demonstrating how In-Database Functions can be used for model training and scoring and comparing the performance of 2 models. The data we are using is a sample dataset and the results and predictions may not be entirely accurate.
</p>



<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>1. Data</b></p>
<p style = 'font-size:16px;font-family:Arial'>This dataset contains data for 10000 patients, half of which were diagnosed with diabetes. It contains 624 columns. We used patients` visit records consisting of diagnoses, procedures, medications and demographics. In addition, we also added a temporal aspect to the medical features. We differentiated between events occurring 1-3 months before diagnosis, 3-6 months, and 6-12 months, before the prediction window. The main aim of the data is to distinguish between healthy people and those who were diagnosed with diabetes, according to "target" column which is set to 0 for non-diabetic and 1 for diabetic.</p>

<p style = 'font-size:16px;font-family:Arial'><a href = 'https://synthea.mitre.org/'>Link to the dataset</a>: This dataset was generated by Synthea for the experimentation purpose and does not reflect the actual population.</p>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>2. Contents:</b></p>
<ol style = 'font-size:16px;font-family:Arial'>
    <li>Configuring the Environment</li>
    <li>Initiate a connection to Vantage</li>
    <li>Analyze the raw data set</li>
    <li>Train and Test a Decision Forest Model</li>
        <ul>
            <li>4.1 Train and Test split using SAMPLE. Splitting the dataset in 80:20 ratio for Train and Test respectively</li>
            <li>4.2 Train a Model</li> 
                <ol style = 'font-size:16px;font-family:Arial'>
                    <li style = 'font-size:16px;font-family:Arial' >Using the TD_DecisionForest and TD_DecisonForestPredict In Database function to predict if the person can have Diabetes or not. So there are only 2 responses '0' and '1'.</li>
                    <li style = 'font-size:16px;font-family:Arial'>Using the TD_GLM and TD_GLMPredict In Database function to predict if the person can have Diabetes or not. So there are only 2 responses '0' and '1'.</li>
            </ol>
            <li>4.3 Evaluate the Model :- Evaluation of the model is done using the TD_ClassificationEvaluator which provides various parameters for the model like Accuracy, Precision ,Recall etc.</li>
        </ul>
    <li>Cleanup</li>
</ol>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>3. Start by connecting to the Vantage system.</b></p>


<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).</p>

In [ ]:
import json
import getpass
import pandas as pd
from teradataml import *
import warnings
warnings.filterwarnings("ignore")

import teradatasql
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
from sklearn import metrics

<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, then use down arrow to go to next cell. Begin running steps with Shift + Enter keys.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=Diabetes_Classification_DFandGLM.ipynb;' UPDATE FOR SESSION; ''')

<b style = 'font-size:20px;font-family:Arial;color:#E37C4D'>4. Getting Data for This Demo
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage.  Here we are downloading the data to local storage which may yield somewhat faster execution, but there could be considerations of available storage. We are only creating local databases and tables as there are 625 columns in table which will be faster in local tables as compared to foreign tables.</p> 
    
<p style = 'font-size:16px;font-family:Arial'><b>“Note:  The data loading part of this demo will be slow because we have large number of  columns (625 columns)."</b></p>    


In [ ]:
%run -i ../run_procedure.py "call get_data('DEMO_DiabetesPrediction_local');"
 # Takes about 3 minutes 30 seconds


<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>5. Analyze the raw data set</b></p>
<!-- <p style = 'font-size:16px;font-family:Arial'>One of the challenges with this data set is that each recording consists of 755 individual metrics.  If this data set were to be used as input to a Decision Forest or other supervised learning algorithm "as is"; this large number of features would cause extraordinary performance degradation for very little gain in accuracy.  Not to mention, wrangling 755 columns adds additional complexity in programming and automation.</p> -->

<p style = 'font-size:16px;font-family:Arial'>Simple SQL query to show the data. Taking just a sample set of data to show case how we can use In-Database Functions.</p>




In [ ]:
query = '''
SELECT TOP 5 * FROM DEMO_DiabetesPrediction.Diabetes_Diagnosis_Data;
'''
df=DataFrame.from_query(query)
df

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>5. Exploratory Data Analysis</b></p>

<p style = 'font-size:16px;font-family:Arial'>With the help of exploratory data analysis (EDA) techniques, we will summarize the main characteristics about the data.
</p>

In [ ]:
query = '''
SELECT * FROM DEMO_DiabetesPrediction.Diabetes_Diagnosis_Data;
'''
DF = DataFrame.from_query(query)
DF

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>5.1. Age Distribution Plot</b></p>

<p style = 'font-size:16px;font-family:Arial'>Following plot shows the density distribution of the people that were diagnosed with diabetes or not based on their age.
</p>

In [ ]:
DF=DF.to_pandas()
plt.rcParams['figure.figsize'] = [8, 2]
Target = {0:'Non-Diabetic',1:'Diabetic'}
for label in Target:
    X = DF[DF["target"] == label]
    sns.distplot(X["dem_age"], hist = False, kde = True,
                 kde_kws = {'shade': True, 'linewidth': 3}, 
                  label = Target[label])

plt.legend(prop={'size': 8}, title = 'Target')
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Density')
plt.show()

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>5.2. Gender Distribution Plot</b></p>

<p style = 'font-size:16px;font-family:Arial'>Following plot highlights the demographics showing the count of males and females.
</p>

In [ ]:
Gender = DF['dem_female'].value_counts().rename(index={0:"Male",1:"Female"})
plt.bar(list(Gender.keys()),Gender.values)
plt.xlabel("Gender")
plt.ylabel("Frequency")
plt.title("Gender Distribution")
plt.show()

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>5.3. Target Distribution Plot</b></p>

<p style = 'font-size:16px;font-family:Arial'>Following plot highlights the count of diabetic and non-diabetic patients.
</p>

In [ ]:
Target = DF['target'].value_counts().rename(index={0:"Non-Diabetic",1:"Diabetic"})
plt.bar(list(Target.keys()),Target.values)
plt.xlabel("Target")
plt.ylabel("Frequency")
plt.title("Target Distribution")
plt.show()

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>5.4. SNOMED Codes Distribution Plot</b></p>

<p style = 'font-size:16px;font-family:Arial'>Following plot highlights how many visits consist of diagnoses, procedures and medications.
</p>

In [ ]:
Diag = DF.filter(regex="^diag").sum().sum()
Proc = DF.filter(regex="^proc").sum().sum()
Drg = DF.filter(regex="^drg").sum().sum()
Dict = {"Diagnosis":Diag,"Procedures":Proc,"Drugs":Drg}
plt.bar(list(Dict.keys()),Dict.values())
plt.xlabel("Medication")
plt.ylabel("Frequency")
plt.title("SNOMED Codes Distribution")
plt.show()

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>6. Feature Selection</b></p>


<p style = 'font-size:16px;font-family:Arial'>Currently we have 622 features excluding ID and target. We used regression-based feature reduction technique to eliminate uninformative features.
</p>

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>6.1. Create Train and Test Dataset</b></p>

<p style = 'font-size:16px;font-family:Arial'>Now that we have our prepared data set, we can perform an abbreviated machine learning workflow:</p>

<ol style = 'font-size:16px;font-family:Arial'>
    <li>Create Train and Test data sets using SAMPLE Clause(80:20 split)</li>
    <li>Train the model</li>
    <li>Evaluate the model using Test data</li>
</ol>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Train and Test split using SAMPLE</b></p>

<p style = 'font-size:16px;font-family:Arial'>Using EXCEPT clause in the second statement ensures a non-intersecting set of data</p>

In [ ]:
query = '''
CREATE MULTISET TABLE diabetes_features_train AS (
    SELECT * FROM  DEMO_DiabetesPrediction.Diabetes_Diagnosis_Data where DEM_age mod 10 >= 1
) WITH DATA
PRIMARY INDEX(MBR_ID)
'''

try:
    execute_sql(query)
except:
    db_drop_table('diabetes_features_train')
    execute_sql(query)

train = DataFrame.from_query('SELECT TOP 5 * FROM diabetes_features_train')
df_count_train = DataFrame.from_query('SELECT target, COUNT(*) as cnt FROM diabetes_features_train GROUP BY target;')
df_count_train

<p style = 'font-size:16px;font-family:Arial'>The output shows the number of people we are considering for each class to train the model – class 1 has Diabetes</p>

In [ ]:
query = '''
CREATE MULTISET TABLE diabetes_features_test AS (
    SELECT * FROM DEMO_DiabetesPrediction.Diabetes_Diagnosis_Data
    EXCEPT
    SELECT * FROM diabetes_features_train
) WITH DATA
PRIMARY INDEX(MBR_ID)
'''

try:
    execute_sql(query)
except:
    db_drop_table('diabetes_features_test')
    execute_sql(query)

test = DataFrame.from_query('SELECT TOP 5 * FROM diabetes_features_test;')
df_count_test = DataFrame.from_query('SELECT target, COUNT(*) as cnt FROM diabetes_features_test GROUP BY target;')
df_count_test

<p style = 'font-size:16px;font-family:Arial'>The output shows the number of people we are considering for each class to validate the model – class 1 has Diabetes</p>

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>6.2. Using TD_GLM with ElasticNet regularization for feature reduction.</b></p>

In [ ]:
%%time
query = '''
CREATE TABLE td_glm_output_selection AS (
SELECT * FROM td_glm (
ON diabetes_features_train
USING
InputColumns('[2:622]')
ResponseColumn('target')
Family('Binomial')
BatchSize(200)
MaxIterNum(100)
RegularizationLambda(0.00175)
Alpha(0.15)
IterNumNoChange(50)
Tolerance(0.0001)
Intercept('true')
LearningRate('optimal')
InitialEta(0.001)
Momentum(0.0)
LocalSGDIterations(0)
) AS dt
) WITH DATA
;
'''

try:
    execute_sql(query)
except:
    db_drop_table('td_glm_output_selection')
    execute_sql(query)

output = DataFrame.from_query('SELECT * FROM td_glm_output_selection;')

In [ ]:
output=output.to_pandas(all_rows=True).reset_index()
output[output["estimate"] != 0].sort_values(by = "attribute")

In [ ]:
output[output["estimate"] == 0]

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>6.3. Storing the selected features in a new table.</b></p>

In [ ]:
Indexes = output[output["estimate"] == 0]["attribute"].to_list()
Total = list(range(2,623)) 
dif1 = np.setdiff1d(Total, Indexes)
dif2 = np.setdiff1d(Indexes, Total)
 
temp3 = np.concatenate((dif1, dif2))
Features = list(temp3)
Features = list(filter(lambda x : x > 0, Features))
Str = ""
for i in range(len(Features)):
    Str += '"{}"'.format(train.columns[Features[i]])
    if i != (len(Features) - 1):
        Str += ","

In [ ]:
query = '''
CREATE MULTISET TABLE selected_dbt_features_train AS (
    select "MBR_ID",{},"target" from diabetes_features_train
) WITH DATA
PRIMARY INDEX(MBR_ID)
'''.format(Str)

try:
    execute_sql(query)
except:
    db_drop_table('selected_dbt_features_train')
    execute_sql(query)

train = DataFrame.from_query('SELECT TOP 5 * FROM selected_dbt_features_train;')
df_count_features_train = DataFrame.from_query('SELECT target, COUNT(*) as cnt FROM selected_dbt_features_train GROUP BY target;')
df_count_features_train

In [ ]:
query = '''
CREATE MULTISET TABLE selected_dbt_features_test AS (
    select "MBR_ID",{},"target" from diabetes_features_test
) WITH DATA
PRIMARY INDEX(MBR_ID)
'''.format(Str)

try:
    execute_sql(query)
except:
    db_drop_table('selected_dbt_features_test')
    execute_sql(query)

test = DataFrame.from_query('SELECT TOP 5 * FROM selected_dbt_features_test;')
df_count_features_test = DataFrame.from_query('SELECT target, COUNT(*) as cnt FROM selected_dbt_features_test GROUP BY target;')
df_count_features_test

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>7. Decision Tree Model</b></p>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>7.1 - Train a Decision Tree Model</b></p>

<p style = 'font-size:16px;font-family:Arial'>The <a href = 'https://docs.teradata.com/search/all?query=TD_DecisionForest&content-lang=en-US'>TD_DecisionForest</a> is an ensemble algorithm used for classification and regression predictive modeling problems. It is an extension of bootstrap aggregation (bagging) of decision trees. </p>

<p style = 'font-size:16px;font-family:Arial'>This function takes the training data as input, as well as the following function parameters</p>
    <ul style = 'font-size:16px;font-family:Arial'>
        <li>InputColumns; list or range of columns used as features (we used an ordinal reference of columns 2:217)</li>
        <li>ResponseColumn; the dependent or target value (we used “class”, the first column)</li>
        <li>TreeType; either CLASSIFICATION or REGRESSION</li>
    <li>Other hyperparameter values detailed in the documentation</li>
        </ul>

In [ ]:
%%time
query = '''
CREATE multiset table DF_table as (
    SELECT * FROM TD_DecisionForest (
    ON selected_dbt_features_train AS INPUTTABLE PARTITION BY ANY
  USING
      InputColumns('[2:217]')
      ResponseColumn('target')
      MaxDepth(16)
      MinNodeSize(1)
      NumTrees(8)
      ModelType('CLASSIFICATION')
      Seed(3)
      Mtry(-1)
      MtrySeed(3)
    ) as dt
) with data;
'''

try:
    execute_sql(query)
except:
    db_drop_table('DF_table')
    execute_sql(query)

<p style = 'font-size:16px;font-family:Arial'>The TD_DecisionForest function produces a model and a JSON representation of the decision tree. Below is explaination for some columns in the JSON tree. The other details can be found at the link <a href = 'https://docs.teradata.com/search/all?query=TD_DecisionForest&content-lang=en-US'>here.</a></p>

</p>
<html>
   <head>
      <style>
         table, th, td {
            border: 1px solid black;
            border-collapse:collapse;
         }
      </style>
   </head>
   <body>
      <table>
         <tr>
            <th>JSON Type</th>
            <th>Description</th>             
         </tr>
         <tr>
            <td>id_</td>
            <td>"Node identifier"</td>
         </tr>
         <tr>
            <td>nodeType_</td> 
            <td>The node type. Possible values: CLASSIFICATION_NODE,CLASSIFICATION_LEAF,REGRESSION_NODE,REGRESSION_LEAF.</td>
         </tr>
         <tr>
            <td>split_</td> 
            <td>The start of JSON item that describes a split in the node.</td>
         </tr> 
         <tr>
            <td>responseCounts_</td> 
            <td>[Classification trees] Number of observations in each class at node identified by id.</td>
         </tr>
         <tr>
            <td>size_</td> 
            <td>Total number of observations at node identified by id.</td>
         </tr> 
         <tr>
            <td>maxDepth_</td> 
            <td>Maximum possible depth of tree, starting from node identified by id. For root node, the
value is max_depth. For leaf nodes, the value is 0. For other nodes, the value is the
maximum possible depth of tree, starting from that node.</td>
         </tr>  
      </table>
   </body>
</html>


<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>7.2 - Evaluate the Model</b></p>

<p style = 'font-size:16px;font-family:Arial'>Execute a testing prediction using the split data above.  Evaluate the model by creating a confusion matrix with the <a href = 'https://docs.teradata.com/search/all?query=TD_ClassificationEvaluator&content-lang=en-US'>TD_ClassificationEvaluator</a> SQL Function.</p>


<ol style = 'font-size:16px;font-family:Arial'>
    <li>Execute <a href = 'https://docs.teradata.com/r/Teradata-VantageTM-Analytics-Database-Analytic-Functions-17.20/Model-Scoring-Functions/DecisionForestPredict'>DecisionForestPredict</a> using the model built above</li>
    <li>Execute <a href = 'https://docs.teradata.com/search/all?query=TD_ClassificationEvaluator&content-lang=en-US'>TD_ClassificationEvaluator</a> and pass the actual classification and the predicted value</li>
</ol>

In [ ]:
query = '''
create multiset table DF_predict_test as (
    select * from TD_DecisionForestPredict (
        ON selected_dbt_features_test AS INPUTTABLE PARTITION BY ANY
        ON DF_table as ModelTable dimension
        using
        IDColumn ('MBR_ID')
        OutputProb ('true')
        Responses ('0', '1')
        Accumulate ('target')
    ) as dt
) with data;
'''

try:
    execute_sql(query)
except:
    db_drop_table('DF_predict_test')
    execute_sql(query)

df_predict=DataFrame.from_query('SELECT * FROM DF_predict_test;')
df_predict

<p style = 'font-size:16px;font-family:Arial'>The TD_DecisionForestPredict function creates probabilities for the prediction made depending on the class and the Id cloumns. The output of the predict function is passed to the Classification Evaluator to get the parameters of the functions.</p>

<p style = 'font-size:16px;font-family:Arial'>DecisionForestPredict outputs the probability that each observation is in the predicted class. To use DecisionForestPredict output as input to ML Engine ROC function, you must first transform it to show the probability that each observation is in the positive class. One way to do this is to change the probability to (1- current probability) when the predicted class is negative.The prediction algorithm compares floating-point numbers. Due to possible inherent data type differences between ML Engine and Analytics Database executions, predictions can differ.</p>


<p style = 'font-size:16px;font-family:Arial'>We create the Confusion Matrix to compare the actual and the Predicted values. Confusion matrix is a very popular measure used while solving classification problems. It can be applied to binary classification as well as for multiclass classification problems. Confusion matrices represent counts from predicted and actual values. It is an N x N matrix used for evaluating the performance of a classification model, where N is the number of target classes.</p>


In [ ]:
plt.rcParams['figure.figsize'] = [6, 6]
df = pd.read_sql('SELECT MBR_ID, cast(target as int) target, cast(prediction as int) prediction FROM DF_predict_test', eng)
cm = confusion_matrix(df['target'], df['prediction'], normalize='all')
cmd = ConfusionMatrixDisplay(cm, display_labels=['DoesNotHaveDiabetes', 'HasDiabetes'])
cmd.plot()
plt.show()

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>7.3 - Use classification Evaluator for DecisionForestPredict</b></p>

<p style = 'font-size:16px;font-family:Arial'>Evaluate the model by creating a confusion matrix with the <a href = 'https://docs.teradata.com/search/all?query=TD_ClassificationEvaluator&content-lang=en-US'>TD_ClassificationEvaluator</a> SQL Function.</p>

<p style = 'font-size:16px;font-family:Arial'>In classification problems, a confusion matrix is used to visualize the performance of a classifier. The confusion matrix contains predicted labels represented across the row-axis and actual labels represented
across the column-axis. Each cell in the confusion matrix corresponds to the count of occurrences of labels
in the test data.</p>

<p style = 'font-size:16px;font-family:Arial'>Apart from accuracy, the secondary output table returns micro, macro, and weighted-averaged metrics of precision, recall, and F1-score values.</p>


In [ ]:
query = '''
SELECT * FROM TD_ClassificationEvaluator(
   ON (select prediction, "target" from DF_predict_test) AS InputTable
   --cast(target as VARCHAR(32000) CHARACTER SET UNICODE NOT CASESPECIFIC) as target from DF_predict_test) AS InputTable
   OUT TABLE OutputTable(additional_metrics_test)
   USING
   ObservationColumn('target')
   PredictionColumn('prediction')
   Labels(0,1)
) AS dt;
'''

try:
    execute_sql(query)
except:
    db_drop_table('additional_metrics_test')
    execute_sql(query)

df_metrics=DataFrame.from_query('SELECT * FROM additional_metrics_test;')
df_metrics

<p style = 'font-size:16px;font-family:Arial'>The above output has the secondary output table that returns micro, macro, and weighted-averaged metrics of precision, recall, and F1-score values.</p>
<table style = 'font-size:16px;font-family:Arial'>
  <tr>
    <th>Column</th>
    <th>Description</th>
  </tr>
  <tr>
    <td>Precision</td>
    <td>The positive predictive value. Refers to the fraction of relevant instances among
the total retrieved instances.</td>
  </tr>
  <tr>
    <td>Recall</td>
    <td>Refers to the fraction of relevant instances retrieved over the total amount of
relevant instances.</td>
  </tr>
  <tr>
    <td>F1</td>
    <td>F1 score, defined as the harmonic mean of the precision and recall.</td>
  </tr>
  <tr>
    <td>Support</td>
    <td>The number of times a label displays in the ObservationColumn.</td>
  </tr>
</table>

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>7.4 - Show AUC-ROC Curve for DecisionForestPredict</b></p>

<p style = 'font-size:16px;font-family:Arial'>The <a href = 'https://docs.teradata.com/search/all?query=TD_ROC&content-lang=en-US'>TD_ROC</a> curve shows the performance of a binary classification model as its discrimination threshold varies. For a range of thresholds, the curve plots the true positive rate against false-positive rate.</p>

<p style = 'font-size:16px;font-family:Arial'>This function accepts a set of prediction-actual pairs as input and calculates the following values for a range of discrimination thresholds.</p>
    <ul style = 'font-size:16px;font-family:Arial'>
        <li>True-positive rate (TPR)</li>
        <li>False-positive rate (FPR)</li>
        <li>The area under the ROC curve (AUC)</li>
        <li>Gini coefficient</li>
        <li>Other details are mentioned in the documentation</li>
    </ul>



In [ ]:
query = '''
select * from TD_ROC(
on DF_predict_test as InputTable
OUT PERMANENT TABLE OutputTable(DF_RocTable)
Using
ProbabilityColumn ('prob_1')
ObservationColumn ('target')
PositiveLabel ('1')
AUC('true')
)As dt;
'''
try:
    execute_sql(query)
except:
    db_drop_table('DF_RocTable')
    execute_sql(query)
DF_AUC = pd.read_sql('SELECT * FROM DF_RocTable', eng)
auc = metrics.auc(DF_AUC['fpr'], DF_AUC['tpr'])
plt.plot(DF_AUC['fpr'],DF_AUC['tpr'],label="AUC="+str(auc))
plt.legend(loc=4)
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("AUC-ROC Curve")
plt.show()

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>8. Generalized Linear Model(GLM)</b></p>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>8.1 - Train a GLM Model</b></p>

<p style = 'font-size:16px;font-family:Arial'>The <a href = 'https://docs.teradata.com/search/all?query=TD_GLM&content-lang=en-US'>Generalized Linear Model (GLM)</a> is an extension of the linear regression model that enables the linear equation to relate to the dependent variables by a link function. The GLM function supports several distribution families and associated link functions. </p>

<p style = 'font-size:16px;font-family:Arial'>This function takes the training data as input, as well as the following function parameters</p>
    <ul style = 'font-size:16px;font-family:Arial'>
        <li>InputColumns; list or range of columns used as features (we used an ordinal reference of columns 2:217)</li>
        <li>ResponseColumn; the dependend or target value (we used “class”, the first column) </li>
        <li>Family; either Binomial or Gaussian</li>
    <li>Other hyperparameter values detailed in the documentation</li>
        </ul>

<p style = 'font-size:16px;font-family:Arial'>Using the TD_GLM, we are training the model on our data.</p>

In [ ]:
%%time
query = '''
CREATE TABLE td_glm_output AS (
SELECT * FROM td_glm (
ON selected_dbt_features_train
USING
InputColumns('[2:217]')
ResponseColumn('target')
Family('Binomial')
BatchSize(200)
MaxIterNum(300)
RegularizationLambda(0.01)
Alpha(0.15)
IterNumNoChange(50)
Tolerance(0.008)
Intercept('true')
LearningRate('optimal')
InitialEta(0.05)
Momentum(0.0)
LocalSGDIterations(0)
) AS dt
) WITH DATA
;
'''

try:
    execute_sql(query)
except:
    db_drop_table('td_glm_output')
    execute_sql(query)

df_glm=DataFrame.from_query('SELECT * FROM td_glm_output;')
df_glm

<p style = 'font-size:16px;font-family:Arial'>The TD_GLM function creates various output predictors and values based on the above parameters passed in the query</p>

<p style = 'font-size:16px;font-family:Arial'>The function output is a trained GLM model which can be input to the TD_GLMPredict function
for prediction. The model also contains model statistics of MSE, Loglikelihood, AIC, and BIC.
Further model evaluation can be done as a post-processing step using functions such as
TD_RegressionEvaluator,TD_ClassificationEvaluator and TD_ROC.</p>


<p style = 'font-size:16px;font-family:Arial;> The TD_DecisionForest function creates a tree as seen in the output above based on the parameters applied in the query. </b></p>



<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>8.2 - Evaluate the Model</b></p>

<p style = 'font-size:16px;font-family:Arial'>Execute a testing prediction using the split data above.  Evaluate the model by creating a confusion matrix with the <a href = 'https://docs.teradata.com/search/all?query=TD_ClassificationEvaluator&content-lang=en-US'>TD_ClassificationEvaluator</a> SQL Function.</p>


<ol style = 'font-size:16px;font-family:Arial'>
    <li>Execute <a href = 'https://docs.teradata.com/search/all?query=TD_GLMPredict&content-lang=en-US'>GLMPredict</a> using the model built above</li>
    <li>Execute <a href = 'https://docs.teradata.com/search/all?query=TD_ClassificationEvaluator&content-lang=en-US'>TD_ClassificationEvaluator</a> and pass the actual classification and the predicted value</li>
</ol>

In [ ]:
query = '''
CREATE TABLE vt_glm_predict AS (
SELECT * from TD_GLMPredict (
ON selected_dbt_features_test AS INPUTTABLE
ON td_glm_output AS ModelTable DIMENSION
USING
IDColumn ('MBR_ID')
Accumulate('target')
OutputProb('true')
Responses ('0','1')
) AS dt
) WITH DATA
;
'''

try:
    execute_sql(query)
except:
    db_drop_table('vt_glm_predict')
    execute_sql(query)

df_glm_predict=DataFrame.from_query('SELECT * FROM vt_glm_predict;')
df_glm_predict

<p style = 'font-size:16px;font-family:Arial'>The TD_GLMPredict function predicts target values (regression) and class labels (classification) for test data using a GLM model trained by the TD_GLM function.Similar to TD_GLM, input features should be standardized, such as using TD_ScaleFit, and TD_ScaleTransform, before using in the function. The function takes only numeric features. The categorical
features must be converted to numeric values prior to prediction.</p>

<p style = 'font-size:16px;font-family:Arial'>Rows with missing (null) values are skipped by the function during prediction. For prediction results evaluation, you can use TD_RegressionEvaluator, TD_ClassificationEvaluator or TD_ROC function as
postprocessing step.</p>


<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>8.3 - Use classification Evaluator for GLMPredict</b></p>

<p style = 'font-size:16px;font-family:Arial'>Evaluate the model by creating a confusion matrix with the <a href = 'https://docs.teradata.com/search/all?query=TD_ClassificationEvaluator&content-lang=en-US'>TD_ClassificationEvaluator</a> SQL Function.</p>



<p style = 'font-size:16px;font-family:Arial'>Since TD_ClassificationEvaluator requires same datatype for prediction and class columns so creating another table with same datatype.</p>

In [ ]:
query = '''CREATE SET TABLE vt_glm_predict_conv ,FALLBACK ,
     CHECKSUM = DEFAULT,
     DEFAULT MERGEBLOCKRATIO,
     MAP = TD_MAP1,
     LOG
     (
      MBR_ID VARCHAR(64000) CHARACTER SET LATIN NOT CASESPECIFIC,
      prediction INT,
      target INT,
            prob_0 FLOAT,
      prob_1 FLOAT

)PRIMARY INDEX ( MBR_ID )
;
'''

try:
    execute_sql(query)
except:
    db_drop_table('vt_glm_predict_conv')
    execute_sql(query)

query = '''insert into vt_glm_predict_conv sel MBR_ID , cast(prediction as int), target , prob_0, prob_1 from vt_glm_predict;'''
out=execute_sql(query)    

<p style = 'font-size:16px;font-family:Arial'>Create CONFUSION MATRIX for the GLM Predict model.</p>

In [ ]:
df_glm = pd.read_sql('SELECT * FROM vt_glm_predict_conv', eng)
cm = confusion_matrix(df_glm['target'], df_glm['prediction']) # , normalize='all'
cmd = ConfusionMatrixDisplay(cm, display_labels=['DoesNotHaveDiabetes', 'HasDiabetes'])
cmd.plot()
plt.show()

In [ ]:
query = '''
SELECT * FROM TD_ClassificationEvaluator(
       ON (select prediction,  target from vt_glm_predict_conv) AS InputTable
       OUT TABLE OutputTable(additional_metrics_test_glm)
       USING
       ObservationColumn('target')
       PredictionColumn('prediction')
       Labels(0,1)
    ) AS dt;
'''

try:
    execute_sql(query)
except:
    db_drop_table('additional_metrics_test_glm')
    execute_sql(query)

df_metrics_glm=DataFrame.from_query('SELECT * FROM additional_metrics_test_glm;')
df_metrics_glm

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>8.4 - Show AUC-ROC Curve for GLMPredict</b></p>

<p style = 'font-size:16px;font-family:Arial'>The <a href = 'https://docs.teradata.com/search/all?query=TD_ROC&content-lang=en-US'>TD_ROC</a> curve shows the performance of a binary classification model as its discrimination threshold varies. For a range of thresholds, the curve plots the true positive rate against false-positive rate.</p>

<p style = 'font-size:16px;font-family:Arial'>This function accepts a set of prediction-actual pairs as input and calculates the following values for a range of discrimination thresholds.</p>
    <ul style = 'font-size:16px;font-family:Arial'>
        <li>True-positive rate (TPR)</li>
        <li>False-positive rate (FPR)</li>
        <li>The area under the ROC curve (AUC)</li>
        <li>Gini coefficient</li>
        <li>Other details are mentioned in the documentation</li>
    </ul>



In [ ]:
query = '''
select * from TD_ROC(
on vt_glm_predict as InputTable
OUT PERMANENT TABLE OutputTable(glm_RocTable)
Using
ProbabilityColumn ('prob_1')
ObservationColumn ('target')
PositiveLabel ('1')
AUC('true')
)As dt;
'''
try:
    execute_sql(query)
except:
    db_drop_table('glm_RocTable')
    execute_sql(query)
glm_AUC = pd.read_sql('SELECT * FROM glm_RocTable', eng)
auc = metrics.auc(glm_AUC['fpr'], glm_AUC['tpr'])
plt.plot(glm_AUC['fpr'],glm_AUC['tpr'],label="AUC="+str(auc))
plt.legend(loc=4)
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("AUC-ROC Curve")
plt.show()

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>9. Comparison of the Metrics generated by the 2 Models. Decision Forest vs GLM</b></p>

In [ ]:
query = '''CREATE MULTISET TABLE metric_union as (select cast('DecisionForest' as VARCHAR(15)) as Model, trim(Trailing '2E'XC from cast(Metric as VARCHAR(18))) as Metric,MetricValue from additional_metrics_test a 
union all 
select 'GLM' as Model ,  trim(Trailing '2E'XC from cast(Metric as VARCHAR(18))) as Metric,MetricValue from additional_metrics_test_glm b
)with data PRIMARY INDEX (Metric)
;
'''

try:
    execute_sql(query)
except:
    db_drop_table('metric_union')
    execute_sql(query)
    
df_chart =DataFrame.from_query('select * from metric_union;')
df_chart=df_chart.to_pandas().reset_index()

In [ ]:
df_chart['Metric'] = df_chart['Metric'].str.replace('\x00', '')
df_pivot = pd.pivot_table(
df_chart,
values="MetricValue",
index="Metric",
columns="Model"
)
# df_pivot=df_pivot.reset_index()
# df_pivot['Metric'] = df_pivot['Metric'].str.replace(r'\x00', '')
#df_chart.plot.bar(x='Metric',y='MetricValue' , legend='model')
ax=df_pivot.plot(kind='bar')
# Get a Matplotlib figure from the axes object for formatting purposes
fig = ax.get_figure()
# Change the plot dimensions (width, height)
fig.set_size_inches(12, 6)
# Change the axes labels
ax.set_xlabel("Metrics")
ax.set_ylabel("Metric Values")
ax.tick_params(axis='x', rotation=45)

<p style = 'font-size:16px;font-family:Arial'>Thus here we have used 2 different models to train and predict the data. The classification evaluator is used to evaluate and compare the models. The Teradata In-Database functions are used for training, prediction and evaluation. In this case since we have sample data the result parameters like the Accuracy, Precision, Recall etc. may not be accurate for both the models, still from the above graph we can conclude that in this case TD_DecisionForest is better than TD_GLM.  


<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>11. Cleanup</b></p>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Work Tables</b></p>

In [ ]:
tables = ['diabetes_features_test', 'diabetes_features_train','td_glm_output_selection','selected_dbt_features_train','DF_table',
          'DF_predict_test','additional_metrics_test','td_glm_output','vt_glm_predict_conv','vt_glm_predict','additional_metrics_test_glm'
         , 'metric_union' ]

# Loop through the list of tables and execute the drop table command for each table
for table in tables:
    try:
        db_drop_table(table_name=table)
    except:
        pass

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Databases and Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_DiabetesPrediction');" 
#Takes 40 seconds

In [ ]:
remove_context()

<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">Copyright © Teradata Corporation - 2023. All Rights Reserved.</footer>